In [1]:
from PIL import Image
import numpy as np

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

2024-06-05 16:31:18.228148: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-05 16:31:18.368049: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
def load_and_pad_image(image_path, target_size):
    img = Image.open(image_path)
    img_array = np.array(img)
    pad_height = max(0, target_size[0] - img_array.shape[0])
    pad_width = max(0, target_size[1] - img_array.shape[1])
    pad_top = pad_height // 2
    pad_bottom = pad_height - pad_top
    pad_left = pad_width // 2
    pad_right = pad_width - pad_left
    img_array = np.pad(img_array, ((pad_top, pad_bottom), (pad_left, pad_right), (0, 0)), mode='constant', constant_values=0)
    return img_array

In [5]:
def load_image_pairs_with_padding(blurred_folder, correct_folder, target_size):
    blurred_images = []
    correct_images = []
    for filename in os.listdir(blurred_folder):
        blurred_path = f'{blurred_folder}{filename}'
        correct_path = f'{correct_folder}{filename}'  # zakładamy, że nazwy plików są takie same
        if os.path.exists(correct_path):
            blurred_img = load_and_pad_image(blurred_path, target_size)
            correct_img = load_and_pad_image(correct_path, target_size)
            blurred_images.append(blurred_img)
            correct_images.append(correct_img)
    return np.array(blurred_images), np.array(correct_images)

In [6]:
import os

In [7]:
# Ścieżki do folderów
correct_images_folder = 'Pictures/'
blurred_images_folder = 'Zgaussowane/'

target_size = (248, 248)

# Załaduj pary obrazów (rozmyte i nierozmyte)
blurred_images, correct_images = load_image_pairs_with_padding(blurred_images_folder, correct_images_folder, target_size)


In [8]:
blurred_images[2,120,4500]

IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed

In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D

In [10]:
input_img = Input(shape=(target_size[0], target_size[1], 3))

In [11]:
# Encoder
x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

In [12]:
# Model
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(blurred_images, correct_images, test_size=0.2, random_state=42)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [15]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [16]:
autoencoder.fit(x_train, y_train, epochs=50, batch_size=16, shuffle=True, steps_per_epoch=100, validation_data=(x_test, y_test))

NameError: name 'x_train' is not defined